## Creating a logistic regression to predict absenteeism

### Import the relevant libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
data_prepro = pd.read_csv('df_preprocessed.csv')

In [3]:
data_prepro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 15 columns):
reason_1                     700 non-null int64
reason_2                     700 non-null int64
reason_3                     700 non-null int64
reason_4                     700 non-null int64
Transportation Expense       700 non-null int64
Distance to Work             700 non-null int64
Age                          700 non-null int64
Daily Work Load Average      700 non-null float64
Body Mass Index              700 non-null int64
Education                    700 non-null int64
Children                     700 non-null int64
Pets                         700 non-null int64
Absenteeism Time in Hours    700 non-null int64
Month                        700 non-null int64
Day                          700 non-null int64
dtypes: float64(1), int64(14)
memory usage: 82.1 KB


### Create the Targets

In [4]:
# targets will be the absenteeism hours
# classified as excessive (1) if it is more than the median value, 0 if less
data_prepro['excess absent'] = np.where(data_prepro['Absenteeism Time in Hours'] >
                                             data_prepro['Absenteeism Time in Hours'].median(), 1, 0)

# by using the median we have implicitly balance the dataset

In [5]:
data_prepro = data_prepro.drop(['Absenteeism Time in Hours', 'Day', 'Daily Work Load Average', 'Distance to Work'], axis=1)

In [6]:
data_prepro['excess absent'].unique()

array([1, 0], dtype=int64)

In [7]:
data_prepro['excess absent'].sum() / len(data_prepro['excess absent'])

# this means around 45% data is one, and around 55% is zero
# usually 60-40 split will work ok in logistic regression
# but not in neural networks
# maybe around 45-55 is the limit for any classification method

0.45571428571428574

In [8]:
targets = np.array(data_prepro['excess absent'])

In [9]:
data_with_targets = data_prepro.copy()

In [10]:
data_with_targets.head()

,reason_1,reason_2,reason_3,reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month,excess absent
0,0,0,0,1,289,33,30,0,2,1,7,1
1,0,0,0,0,118,50,31,0,1,0,7,0
2,0,0,0,1,179,38,31,0,0,0,7,0
3,1,0,0,0,279,39,24,0,2,0,7,1
4,0,0,0,1,289,33,30,0,2,1,7,0


In [11]:
data_with_targets is data_prepro

False

### Select the inputs for the regression

In [12]:
data_with_targets.shape

(700, 12)

In [13]:
data_with_targets.iloc[:,:-1]

,reason_1,reason_2,reason_3,reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month
0,0,0,0,1,289,33,30,0,2,1,7
1,0,0,0,0,118,50,31,0,1,0,7
2,0,0,0,1,179,38,31,0,0,0,7
3,1,0,0,0,279,39,24,0,2,0,7
4,0,0,0,1,289,33,30,0,2,1,7
5,0,0,0,1,179,38,31,0,0,0,7
6,0,0,0,1,361,28,27,0,1,4,7
7,0,0,0,1,260,36,23,0,4,0,7
8,0,0,1,0,155,34,25,0,2,0,7
9,0,0,0,1,235,37,29,1,1,1,7


In [14]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

### Standardize the data (again)

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

In [16]:
class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    def fit(self,X,y=None):
        self.scaler.fit(X[self.columns],y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self,X,y=None,copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]),columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled,X_scaled], axis=1)[init_col_order]

In [17]:
unscaled_inputs.columns.values

array(['reason_1', 'reason_2', 'reason_3', 'reason_4',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month'], dtype=object)

In [47]:
columns_to_omit = ['reason_1', 'reason_2', 'reason_3', 'reason_4', 'Education']

columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [48]:
scaler = CustomScaler(columns_to_scale)

In [49]:
scaler.fit(unscaled_inputs)

C:\Users\Public\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


CustomScaler(columns=['Transportation Expense', 'Age', 'Body Mass Index', 'Children', 'Pets', 'Month'],
       copy=None, with_mean=None, with_std=None)

In [50]:
scaled_inputs = scaler.transform(unscaled_inputs)
scaled_inputs

C:\Users\Public\Anaconda2\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


,reason_1,reason_2,reason_3,reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month
0,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.182726
1,0,0,0,0,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690,0.182726
2,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.182726
3,1,0,0,0,0.854936,0.405184,-0.643782,0,0.880469,-0.589690,0.182726
4,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.182726
5,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.182726
6,0,0,0,1,2.092381,-1.320435,0.061825,0,-0.019280,2.843016,0.182726
7,0,0,0,1,0.568211,-0.065439,-0.878984,0,2.679969,-0.589690,0.182726
8,0,0,1,0,-1.016322,-0.379188,-0.408580,0,0.880469,-0.589690,0.182726
9,0,0,0,1,0.190942,0.091435,0.532229,1,-0.019280,0.268487,0.182726


In [51]:
scaled_inputs.shape

(700, 11)

### Split the data into train & test and shuffle

#### Import the relevant module

In [52]:
from sklearn.model_selection import train_test_split

#### Split

In [53]:
# train_test_split(scaled_inputs, targets, train_size=0.9, shuffle=True, random_state=20)

# split into 4: inputs for training and test, then targets for training and test
# 0.9 or 90% data will be the train data
# default shuffle value is True -> rerun may differ the data also the accuracy!!
# random_state give the option so that the shuffle proceed in the same 'random' way

In [54]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state=20)

C:\Users\Public\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [55]:
print(x_train.shape, y_train.shape)  # 80%

(560, 11) (560,)


In [56]:
print(x_test.shape, y_test.shape)    # 20%

(140, 11) (140,)


### Logistic regression with sklearn

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### Training the model

In [58]:
reg = LogisticRegression()

In [59]:
reg.fit(x_train, y_train)

C:\Users\Public\Anaconda2\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [60]:
reg.score(x_train, y_train)

0.775

### Manually check the accuracy

In [61]:
# accuracy means that x% of the model output match the targets

model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [62]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [63]:
np.sum(model_outputs==y_train)/model_outputs.shape[0]

# same as reg.score() function above

0.775

### Finding the intercept and coefficients

In [64]:
reg.intercept_

array([-1.46547112])

In [65]:
reg.coef_

array([[ 2.62749942,  0.86338637,  2.96050661,  0.66390745,  0.59979822,
        -0.17245127,  0.27568526, -0.23452541,  0.34249662, -0.2775137 ,
         0.15493732]])

In [66]:
unscaled_inputs.columns.values

array(['reason_1', 'reason_2', 'reason_3', 'reason_4',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month'], dtype=object)

In [67]:
summary_table = pd.DataFrame(data = reg.coef_.T, index = unscaled_inputs.columns.values, columns=['Feature coef'])
summary_table

,Feature coef
reason_1,2.627499
reason_2,0.863386
reason_3,2.960507
reason_4,0.663907
Transportation Expense,0.599798
Age,-0.172451
Body Mass Index,0.275685
Education,-0.234525
Children,0.342497
Pets,-0.277514


In [68]:
summary_table.loc['Intercept'] = reg.intercept_[0]
summary_table

,Feature coef
reason_1,2.627499
reason_2,0.863386
reason_3,2.960507
reason_4,0.663907
Transportation Expense,0.599798
Age,-0.172451
Body Mass Index,0.275685
Education,-0.234525
Children,0.342497
Pets,-0.277514


### Interpreting the coefficient

In [69]:
summary_table['Odds_ratio'] = np.exp(summary_table)

In [70]:
summary_table.sort_values('Odds_ratio', ascending=False)

# a feature is not particularly important if
# - its coefficient is around 0: no matter the feature value will be multiplied by 0
# - and if its odds ratio is around 1: for a unit change in standardize feature ,
#     the odds increase by to the multiple of odds ratio, 1 means no change
# consider to drop them

,Feature coef,Odds_ratio
reason_3,2.960507,19.307751
reason_1,2.627499,13.839121
reason_2,0.863386,2.371177
reason_4,0.663907,1.942367
Transportation Expense,0.599798,1.821751
Children,0.342497,1.408460
Body Mass Index,0.275685,1.317433
Month,0.154937,1.167585
Age,-0.172451,0.841599
Education,-0.234525,0.790946


### Backward elimination

In [71]:
# By droping 'Day', 'Daily Work Load Average', 'Distance to Work'
# we get better accuracy, which means those features do not mean much

### Testing the models

In [73]:
reg.score(x_test, y_test)

# test with whole new data (by definition) will always drop the accuracy
# often the test accuracy is 10-20% lower than the train accuracy(due to overfiting)
# if the number is higher either it is lucky or a mistake

0.75

In [76]:
# sklearn.linear_model.LogisticRegression.predict_proba(x)
# returns the probability estimates for all possible outputs
# logsitic regresion models calculate these probability in the background <0.5 it place 0, else 1

predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.71221976, 0.28778024],
       [0.58760009, 0.41239991],
       [0.44337438, 0.55662562],
       [0.77903962, 0.22096038],
       [0.08458343, 0.91541657],
       [0.33103371, 0.66896629],
       [0.29792496, 0.70207504],
       [0.12956221, 0.87043779],
       [0.78307821, 0.21692179],
       [0.74708659, 0.25291341],
       [0.49514969, 0.50485031],
       [0.22640297, 0.77359703],
       [0.07030984, 0.92969016],
       [0.73504052, 0.26495948],
       [0.30533085, 0.69466915],
       [0.55035881, 0.44964119],
       [0.55027426, 0.44972574],
       [0.53930442, 0.46069558],
       [0.40117774, 0.59882226],
       [0.05320682, 0.94679318],
       [0.69874615, 0.30125385],
       [0.77903962, 0.22096038],
       [0.41634563, 0.58365437],
       [0.41634563, 0.58365437],
       [0.2412915 , 0.7587085 ],
       [0.74317087, 0.25682913],
       [0.51065194, 0.48934806],
       [0.85703303, 0.14296697],
       [0.19934235, 0.80065765],
       [0.77903962, 0.22096038],
       [0.

### Save the model

In [77]:
import pickle

# pickle [module] is a Python module used to convert a python object into a character stream

In [78]:
with open('model', 'wb') as file:  #filename, writebyte
    pickle.dump(reg, file)         #dump means save, need argument object to be dumped

In [80]:
with open('scaler', 'wb') as file:
    pickle.dump(scaler, file)